In [ ]:
!pip install geopandas
!pip install geopy
!pip install folium

In [ ]:
import folium 
import csv
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
df = pd.read_csv("faridabad_wards.csv", encoding= 'unicode_escape')

print(df)

    Ward No.                 Ward
0          1       Khori Jamalpur
1          2                  Kot
2          3               Sirohi
3          4              Alampur
4          5          Tikri Khera
5          6                Dhauj
6          7               Mangar
7          8                Pavta
8          9        Fatehpur Taga
9         10             Sarurpur
10        11               Nekpur
11        12             Madalpur
12        13         Kheri Gujran
13        14    Gothra Mohabtabad
14        15                 Pali
15        16                Kavra
16        17         Rajpur Kalan
17        18            Bhanskola
18        19               Lalpur
19        20             Dadasiya
20        21              Bhupani
21        22              Palwali
22        23              Tikwali
23        24               Tilpat
24        25              Bhatola
25        26               Pakhal
26        27          Kheri Kalan
27        28       Baroli/ Badoli
28        29  

In [ ]:
df['Country'] = 'India'
df.head()

,Ward No.,Ward,Country
0,1,Khori Jamalpur,India
1,2,Kot,India
2,3,Sirohi,India
3,4,Alampur,India
4,5,Tikri Khera,India


In [ ]:
df['State'] = 'Faridabad'
df.head()

,Ward No.,Ward,Country,State
0,1,Khori Jamalpur,India,Faridabad
1,2,Kot,India,Faridabad
2,3,Sirohi,India,Faridabad
3,4,Alampur,India,Faridabad
4,5,Tikri Khera,India,Faridabad


In [ ]:
df["location"] = df['Ward'] +", " + df['State'] +", " + df['Country']
df.head()

,Ward No.,Ward,Country,State,location
0,1,Khori Jamalpur,India,Faridabad,"Khori Jamalpur, Faridabad, India"
1,2,Kot,India,Faridabad,"Kot, Faridabad, India"
2,3,Sirohi,India,Faridabad,"Sirohi, Faridabad, India"
3,4,Alampur,India,Faridabad,"Alampur, Faridabad, India"
4,5,Tikri Khera,India,Faridabad,"Tikri Khera, Faridabad, India"


In [ ]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
import numpy as np
   
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = []       
  
# function to find the coordinate 
# of a given city  
def findGeocode(location): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="myGeocoder")
        
        return geolocator.geocode(location) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(location) 
# each value from city column 
# will be fetched and sent to 
# function find_geocode    
for i in (df["location"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 

In [ ]:
df["Longitude"] = longitude 
df["Latitude"] = latitude 

df.head()

,Ward No.,Ward,Country,State,location,Longitude,Latitude
0,1,Khori Jamalpur,India,Faridabad,"Khori Jamalpur, Faridabad, India",77.160005,28.330003
1,2,Kot,India,Faridabad,"Kot, Faridabad, India",77.160005,28.330003
2,3,Sirohi,India,Faridabad,"Sirohi, Faridabad, India",77.160005,28.330003
3,4,Alampur,India,Faridabad,"Alampur, Faridabad, India",77.160005,28.330003
4,5,Tikri Khera,India,Faridabad,"Tikri Khera, Faridabad, India",77.160005,28.330003


In [ ]:
df.to_csv("faridabad.csv")

In [ ]:
map_hooray = folium.Map(location=[28.330003, 77.160005], zoom_start = 13) 

# Ensure you're handing it floats
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray

In [ ]:
df['Latitude']= df['Latitude'].fillna(0)
df['Longitude']= df['Longitude'].fillna(0)
df.head()

,Ward No.,Ward,Country,State,location,Longitude,Latitude
0,1,Khori Jamalpur,India,Faridabad,"Khori Jamalpur, Faridabad, India",0.000000,0.000000
1,2,Kot,India,Faridabad,"Kot, Faridabad, India",77.167555,28.333293
2,3,Sirohi,India,Faridabad,"Sirohi, Faridabad, India",77.180124,28.321093
3,4,Alampur,India,Faridabad,"Alampur, Faridabad, India",0.000000,0.000000
4,5,Tikri Khera,India,Faridabad,"Tikri Khera, Faridabad, India",0.000000,0.000000


In [ ]:
latitude = 28.330003
longitude = 77.160005
faridabad_map = folium.Map(location=[latitude, longitude], zoom_start=13)

In [ ]:
for lat, lon, ward in zip(df['Latitude'], df['Longitude'], df['Ward']):
    folium.CircleMarker(
        [lat, lon],
        popup = ('Ward: ' + str(ward).capitalize()
                ),
        color='blue',
        key_on = ward,
        threshold_scale=[0,1,2,3],
        fill=True,
        fill_opacity=0.7
        ).add_to(faridabad_map)
faridabad_map